# Longest Common Subsequence

Заданы две строки s1 и s2. Требуется найти длину самой длинной общей подпоследовательности. Если общей подпоследовательности нет - выдать 0.

Подпоследовательность - это строка, сгенерированная из исходной строки путем удаления 0 или более символов и без изменения относительного порядка остальных символов.

Например, подпоследовательностями “ABC” являются “”, “A”, “B”, “C”, “AB”, “AC”, “BC” и “ABC”.

В общем случае, строка длины $n$ имеет $2^n$ подпоследовательностей.

"Наивный" алгоритм. Время - $O(2^{\min(m,n)})$, память - $O(\min(m, n))$

In [1]:
# A Naive recursive implementation of LCS problem

# Returns length of LCS for s1[1:i], s2[1:j]
function lcs(i, j, s1, s2)
    if (i == 0 || j == 0)
        0
    elseif (s1[i] == s2[j])
        1 + lcs(i - 1, j - 1, s1, s2)
    else
        max(lcs(i, j - 1, s1, s2), lcs(i - 1, j, s1, s2))
    end
end

lcs(s1, s2) = lcs(length(s1), length(s2), s1, s2)

lcs (generic function with 2 methods)

In [2]:
using Test

In [3]:

@test lcs("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
@test lcs("AGGTAB", "GXTXAYB") == 4
@test lcs("ABC", "CBA") == 1

Test Passed

## Мемоизация

Алгоритм с мемоизацией. Время - $O(m * n)$, память - $O(m * n)$.

In [4]:
# A recursive solution of LCS problem with memoization

function lcs_memo(d, i, j, s1, s2)
    haskey(d, (i, j)) && return d[(i, j)]

    if (i == 0 || j == 0)
        d[(i, j)] = 0
    else
        if (s1[i] == s2[j])
            d[(i, j)] = 1 + lcs_memo(d, i - 1, j - 1, s1, s2)
        else
            d[(i, j)] = max(lcs_memo(d, i, j - 1, s1, s2), lcs_memo(d, i - 1, j, s1, s2))
        end
    end
end

lcs_memo (generic function with 1 method)

In [5]:
function lcs_memo(s1, s2)
    d = Dict{Tuple{Int,Int},Int}()
    lcs_memo(d, length(s1), length(s2), s1, s2)
end

lcs_memo (generic function with 2 methods)

In [6]:
@test lcs_memo("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
@test lcs_memo("AGGTAB", "GXTXAYB") == 4
@test lcs_memo("ABC", "CBA") == 1

Test Passed

## Специализация по `i` и `j`

- Peter Thiemann. 1999. **Combinators for program generation.** J. Funct. Program. 9, 5 (September 1999), 483–525. https://doi.org/10.1017/S0956796899003469

- Kedar Swadi, Walid Taha, Oleg Kiselyov. 2005. **Staging dynamic programming algorithms**. Unpublished manuscript (April 2005), available from: <http://www.cs.rice.edu/~taha/publications.html>.

- Yukiyoshi Kameyama, Oleg Kiselyov, and Chung-chieh Shan. 2009. **Shifting the stage: staging with delimited control.** In Proceedings of the 2009 ACM SIGPLAN workshop on Partial evaluation and program manipulation (PEPM '09). Association for Computing Machinery, New York, NY, USA, 111–120. <https://doi.org/10.1145/1480945.1480962>

- Oleg Kiselyov. 2010. **Delimited control in OCaml, abstractly and concretely: system description.** In Proceedings of the 10th international conference on Functional and Logic Programming (FLOPS'10). Springer-Verlag, Berlin, Heidelberg, 304–320. https://doi.org/10.1007/978-3-642-12251-4_22

In [7]:
using MacroTools: prettify

"Наивный" генератор. Получается одно громадное выражение, в котором есть совпадающие подвыражения.

In [8]:
function lcs_gen_n(i, j)
    if (i == 0 || j == 0)
        0
    else
        quote
            if (s1[$i] == s2[$j])
                1 + $(lcs_gen_n(i - 1, j - 1))
            else
                max($(lcs_gen_n(i, j - 1)), $(lcs_gen_n(i - 1, j)))
            end
        end
    end
end

lcs_gen_n (generic function with 1 method)

In [9]:
lcs_gen_n(2, 3) |> prettify

:(if s1[2] == s2[3]
      1 + if s1[1] == s2[2]
              1 + 0
          else
              max(if s1[1] == s2[1]
                      1 + 0
                  else
                      max(0, 0)
                  end, 0)
          end
  else
      max(if s1[2] == s2[2]
              1 + if s1[1] == s2[1]
                      1 + 0
                  else
                      max(0, 0)
                  end
          else
              max(if s1[2] == s2[1]
                      1 + 0
                  else
                      max(0, if s1[1] == s2[1]
                              1 + 0
                          else
                              max(0, 0)
                          end)
                  end, if s1[1] == s2[2]
                      1 + 0
                  else
                      max(if s1[1] == s2[1]
                              1 + 0
                          else
                              max(0, 0)
                          end, 0)
                  

Не очень хорошо, когда выражения получаются слишком большими. (Это может "взорвать" компилятор, и читать трудно.)

Посему, введём переменные для промежуточных результатов.

Ну, а ещё - введение переменных - это подготовка мемоизации.

## Присваивания для промежуточных результатов

In [10]:
ij2var(i, j) = Symbol("r_", i, "_", j)

ij2var (generic function with 1 method)

In [11]:
function lcs_gen_impl_v(i, j)
    if (i == 0 || j == 0)
        0
    else
        r0 = ij2var(i - 1, j - 1)
        r1 = ij2var(i, j - 1)
        r2 = ij2var(i - 1, j)
        quote
            if (s1[$i] == s2[$j])
                $r0 = $(lcs_gen_impl_v(i - 1, j - 1))
                1 + $r0
            else
                $r1 = $(lcs_gen_impl_v(i, j - 1))
                $r2 = $(lcs_gen_impl_v(i - 1, j))
                max($r1, $r2)
            end
        end
    end
end

lcs_gen_impl_v (generic function with 1 method)

In [12]:
lcs_gen_impl_v(2, 3) |> prettify

:(if s1[2] == s2[3]
      r_1_2 = if s1[1] == s2[2]
              r_0_1 = 0
              1 + r_0_1
          else
              r_1_1 = if s1[1] == s2[1]
                      r_0_0 = 0
                      1 + r_0_0
                  else
                      r_1_0 = 0
                      r_0_1 = 0
                      max(r_1_0, r_0_1)
                  end
              r_0_2 = 0
              max(r_1_1, r_0_2)
          end
      1 + r_1_2
  else
      r_2_2 = if s1[2] == s2[2]
              r_1_1 = if s1[1] == s2[1]
                      r_0_0 = 0
                      1 + r_0_0
                  else
                      r_1_0 = 0
                      r_0_1 = 0
                      max(r_1_0, r_0_1)
                  end
              1 + r_1_1
          else
              r_2_1 = if s1[2] == s2[1]
                      r_1_0 = 0
                      1 + r_1_0
                  else
                      r_2_0 = 0
                      r_1_1 = if s1[1] == s2[1]
        

Недостаток сгенерированной программы в том, что одни и те же выражения повторяются в программе несколько раз.

In [13]:
@generated function lcs_gen_v(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    lcs_gen_impl_v(i, j)
end

lcs_gen_v (generic function with 1 method)

In [14]:
function lcs_gen_v(s1, s2)
    lcs_gen_v(Val(length(s1)), Val(length(s2)), s1, s2)
end

lcs_gen_v (generic function with 2 methods)

In [15]:
@test lcs_gen_v("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
# @test lcs_gen_v("AGGTAB", "GXTXAYB") == 4
@test lcs_gen_v("ABC", "CBA") == 1

Test Passed

Можно вынести присваивания из-под `if (s1[i] == s2[j])`. Тогда будут выполняться лишние вычисления, но асимптотика от этого не изменится.

In [16]:
function lcs_gen_impl_u(i, j)
    if (i == 0 || j == 0)
        0
    else
        r0 = ij2var(i - 1, j - 1)
        r1 = ij2var(i, j - 1)
        r2 = ij2var(i - 1, j)
        quote
            if (s1[$i] == s2[$j])
                $r0 = $(lcs_gen_impl_u(i - 1, j - 1))
                1 + $r0
            else
                $r1 = $(lcs_gen_impl_u(i, j - 1))
                $r2 = $(lcs_gen_impl_u(i - 1, j))
                max($r1, $r2)
            end
        end
    end
end

lcs_gen_impl_u (generic function with 1 method)

In [17]:
lcs_gen_impl_u(2, 3) |> prettify

:(if s1[2] == s2[3]
      r_1_2 = if s1[1] == s2[2]
              r_0_1 = 0
              1 + r_0_1
          else
              r_1_1 = if s1[1] == s2[1]
                      r_0_0 = 0
                      1 + r_0_0
                  else
                      r_1_0 = 0
                      r_0_1 = 0
                      max(r_1_0, r_0_1)
                  end
              r_0_2 = 0
              max(r_1_1, r_0_2)
          end
      1 + r_1_2
  else
      r_2_2 = if s1[2] == s2[2]
              r_1_1 = if s1[1] == s2[1]
                      r_0_0 = 0
                      1 + r_0_0
                  else
                      r_1_0 = 0
                      r_0_1 = 0
                      max(r_1_0, r_0_1)
                  end
              1 + r_1_1
          else
              r_2_1 = if s1[2] == s2[1]
                      r_1_0 = 0
                      1 + r_1_0
                  else
                      r_2_0 = 0
                      r_1_1 = if s1[1] == s2[1]
        

## Распространение констант

Теперь недостаток в том, что одно и то же выражение вычисляется и присваивается несколько раз.

Кроме того, генерируются переменные, которым присваиваются константы.

Можно решить эти две проблемы, заведя словарь, в котором будет регистрироваться, какие подвыражения уже вычислялись, а какие - нет.

В результате получим асимптотику: время - $O(m * n)$, память - $O(m * n)$.

In [18]:
struct Ass!{xs2var, opt}
    es::Vector{Expr}
    d::Dict{Symbol, Any}
    Ass!{xs2var, opt}() where {xs2var, opt} = new(Expr[], Dict{Symbol, Any}())
end

In [19]:
gen_ass!(es, s::Symbol, c::Int64) = c
gen_ass!(es, s::Symbol, u::Symbol) = u

function gen_ass!(es, s::Symbol, e::Expr)
    push!(es, :($s = $e))
    s
end

function (a::Ass!{xs2var, opt})(xs, e) where {xs2var, opt}
    s = xs2var(xs...)
    haskey(a.d, s) && return a.d[s]
    a.d[s] = gen_ass!(a.es, s, opt(e))
end

In [20]:
function lcs_gen_impl!(a::Ass!, i, j)
    if (i == 0 || j == 0)
        a((i, j), 0)
    else
        a((i, j), :(
            if s1[$i] == s2[$j]
                1 + $(lcs_gen_impl!(a, i - 1, j - 1))
            else
                max($(lcs_gen_impl!(a, i, j - 1)),
                    $(lcs_gen_impl!(a, i - 1, j)))
            end
        ))
    end
end

lcs_gen_impl! (generic function with 1 method)

In [21]:
function lcs_gen_impl(a, i, j)
    r = lcs_gen_impl!(a, i, j)
    :($(a.es...); $r)
end

lcs_gen_impl (generic function with 1 method)

In [22]:
lcs_gen_impl(Ass!{ij2var, identity}(), 2, 3) |> prettify

quote
    r_1_1 = if s1[1] == s2[1]
            1 + 0
        else
            max(0, 0)
        end
    r_1_2 = if s1[1] == s2[2]
            1 + 0
        else
            max(r_1_1, 0)
        end
    r_2_1 = if s1[2] == s2[1]
            1 + 0
        else
            max(0, r_1_1)
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_1_3 = if s1[1] == s2[3]
            1 + 0
        else
            max(r_1_2, 0)
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    r_2_3
end

In [23]:
@generated function lcs_gen_c(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    lcs_gen_impl(Ass!{ij2var,identity}(), i, j)
end

lcs_gen_c(s1, s2) = lcs_gen_c(Val(length(s1)), Val(length(s2)), s1, s2)

lcs_gen_c (generic function with 2 methods)

In [24]:
@test lcs_gen_c("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
@test lcs_gen_c("AGGTAB", "GXTXAYB") == 4
@test lcs_gen_c("ABC", "CBA") == 1

Test Passed

## Упрощение выражений через переписывание (Metatheory.jl)

In [25]:
using Metatheory, Metatheory.Rewriters

In [26]:
opt_rules = @theory x y begin

    x::Int64 + y::Int64 => x + y

    max(0, y) --> y
    max(x, 0) --> x

end;

In [27]:
strategy = (#= Fixpoint ∘ =# Postwalk ∘ Chain)
opt_expr(e) = strategy(opt_rules)(e)

opt_expr (generic function with 1 method)

In [28]:
@test opt_expr(:(2 + 3)) == 5
@test opt_expr(:(max(10, 0))) == 10
@test opt_expr(:(max(0, 20))) == 20
@test opt_expr(:(true ? 1 + 2 : 3 + 4)) == :(true ? 3 : 7)

Test Passed

In [29]:
lcs_gen_impl(Ass!{ij2var, opt_expr}(), 2, 3) |> prettify

quote
    r_1_1 = if s1[1] == s2[1]
            1
        else
            0
        end
    r_1_2 = if s1[1] == s2[2]
            1
        else
            r_1_1
        end
    r_2_1 = if s1[2] == s2[1]
            1
        else
            r_1_1
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_1_3 = if s1[1] == s2[3]
            1
        else
            r_1_2
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    r_2_3
end

In [30]:
@generated function lcs_gen_o(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    lcs_gen_impl(Ass!{ij2var,opt_expr}(), i, j)
end

lcs_gen_o(s1, s2) = lcs_gen_o(Val(length(s1)), Val(length(s2)), s1, s2)

lcs_gen_o (generic function with 2 methods)

In [31]:
@test lcs_gen_o("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
@test lcs_gen_o("AGGTAB", "GXTXAYB") == 4
@test lcs_gen_o("ABC", "CBA") == 1

Test Passed

## Использование комбинаторов для генерации программ

- Можно отделить мемоизацию от генерации ("разделение аспектов").
- Генератор можно заставить работать как с мемоизацией, так и без неё.
- Реализацию мемоизации можно использовать для разных генераторов.

### Оператор неподвижной точки

In [32]:
function lcs_gen_step(self, i, j)
    if (i == 0 || j == 0)
        0
    else
        :(
            if s1[$i] == s2[$j]
                1 + $(self(i - 1, j - 1))
            else
                max($(self(i, j - 1)), $(self(i - 1, j)))
            end
        )
    end
end

lcs_gen_step (generic function with 1 method)

In [33]:
function fix_gen!(es, key, step, xs...)
    e = step((ys...) -> fix_gen!(es, key, step, ys...), xs...)
    r = key(xs...)
    push!(es, :($r = $e))
    r
end

function fix_gen(key, step, xs...)
    es = Expr[]
    e = fix_gen!(es, key, step, xs...)
    quote $(es...); $e end
end

fix_gen (generic function with 1 method)

In [34]:
fix_gen(ij2var, lcs_gen_step, 2, 3) |> prettify

quote
    r_0_1 = 0
    r_0_0 = 0
    r_1_0 = 0
    r_0_1 = 0
    r_1_1 = if s1[1] == s2[1]
            1 + r_0_0
        else
            max(r_1_0, r_0_1)
        end
    r_0_2 = 0
    r_1_2 = if s1[1] == s2[2]
            1 + r_0_1
        else
            max(r_1_1, r_0_2)
        end
    r_0_0 = 0
    r_1_0 = 0
    r_0_1 = 0
    r_1_1 = if s1[1] == s2[1]
            1 + r_0_0
        else
            max(r_1_0, r_0_1)
        end
    r_1_0 = 0
    r_2_0 = 0
    r_0_0 = 0
    r_1_0 = 0
    r_0_1 = 0
    r_1_1 = if s1[1] == s2[1]
            1 + r_0_0
        else
            max(r_1_0, r_0_1)
        end
    r_2_1 = if s1[2] == s2[1]
            1 + r_1_0
        else
            max(r_2_0, r_1_1)
        end
    r_0_1 = 0
    r_0_0 = 0
    r_1_0 = 0
    r_0_1 = 0
    r_1_1 = if s1[1] == s2[1]
            1 + r_0_0
        else
            max(r_1_0, r_0_1)
        end
    r_0_2 = 0
    r_1_2 = if s1[1] == s2[2]
            1 + r_0_1
        else
            max(r_1_1, r_0_2)
     

### Вынос мемоизации в комбинатор

In [35]:
function ass!(es, d, s::Symbol, c::Int64)
    d[s] = c
end

function ass!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function ass!(es, d, s::Symbol, e)
    haskey(d, s) && return
    push!(es, :($s = $e))
    d[s] = s
end

ass! (generic function with 3 methods)

In [36]:
function fix_gen_memo!(d, es, key, step, xs...)
    r = key(xs...)
    haskey(d, r) && return d[r]
    e = step((ys...) -> fix_gen_memo!(d, es, key, step, ys...), xs...)
    push!(es, :($r = $e))
    d[r] = r
end

function fix_gen_memo(key, step, xs...)
    es = Expr[]
    e = fix_gen_memo!(Dict(), es, key, step, xs...)
    :($(es...); $e)
end

fix_gen_memo (generic function with 1 method)

In [37]:
fix_gen_memo(ij2var, lcs_gen_step, 2, 3) |> prettify

quote
    r_0_1 = 0
    r_0_0 = 0
    r_1_0 = 0
    r_1_1 = if s1[1] == s2[1]
            1 + r_0_0
        else
            max(r_1_0, r_0_1)
        end
    r_0_2 = 0
    r_1_2 = if s1[1] == s2[2]
            1 + r_0_1
        else
            max(r_1_1, r_0_2)
        end
    r_2_0 = 0
    r_2_1 = if s1[2] == s2[1]
            1 + r_1_0
        else
            max(r_2_0, r_1_1)
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_0_3 = 0
    r_1_3 = if s1[1] == s2[3]
            1 + r_0_2
        else
            max(r_1_2, r_0_3)
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    r_2_3
end

### Распространение констант

In [38]:
function fix_gen_memo_c!(d, es, key, step, xs...)
    r = key(xs...)
    haskey(d, r) && return d[r]
    e = step((ys...) -> fix_gen_memo_c!(d, es, key, step, ys...), xs...)
    ass!(es, d, r, e)
end

function fix_gen_memo_c(key, step, xs...)
    es = Expr[]
    e = fix_gen_memo_c!(Dict(), es, key, step, xs...)
    quote $(es...); $e end
end

fix_gen_memo_c (generic function with 1 method)

In [39]:
fix_gen_memo_c(ij2var, lcs_gen_step, 2, 3) |> prettify

quote
    r_1_1 = if s1[1] == s2[1]
            1 + 0
        else
            max(0, 0)
        end
    r_1_2 = if s1[1] == s2[2]
            1 + 0
        else
            max(r_1_1, 0)
        end
    r_2_1 = if s1[2] == s2[1]
            1 + 0
        else
            max(0, r_1_1)
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_1_3 = if s1[1] == s2[3]
            1 + 0
        else
            max(r_1_2, 0)
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    r_2_3
end

### Оптимизация выражений

In [40]:
function fix_gen_memo_o!(d, es, key, step, xs...)
    r = key(xs...)
    haskey(d, r) && return d[r]
    e = step((ys...) -> fix_gen_memo_o!(d, es, key, step, ys...), xs...)
    ass!(es, d, r, opt_expr(e))
end

function fix_gen_memo_o(key, step, xs...)
    es = Expr[]
    e = fix_gen_memo_o!(Dict(), es, key, step, xs...)
    quote $(es...); $e end
end

fix_gen_memo_o (generic function with 1 method)

In [41]:
fix_gen_memo_o(ij2var, lcs_gen_step, 2, 3) |> prettify

quote
    r_1_1 = if s1[1] == s2[1]
            1
        else
            0
        end
    r_1_2 = if s1[1] == s2[2]
            1
        else
            r_1_1
        end
    r_2_1 = if s1[2] == s2[1]
            1
        else
            r_1_1
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_1_3 = if s1[1] == s2[3]
            1
        else
            r_1_2
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    r_2_3
end

In [42]:
@generated function lcs_fix_gen_o(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    fix_gen_memo_o(ij2var, lcs_gen_step, i, j)
end

lcs_fix_gen_o (generic function with 1 method)

In [43]:
function lcs_fix_gen_o(s1, s2)
    lcs_fix_gen_o(Val(length(s1)), Val(length(s2)), s1, s2)
end

lcs_fix_gen_o (generic function with 2 methods)

In [44]:
@test lcs_fix_gen_o("ABC", "ACD") == 2
# # The longest common subsequence is “GTAB”.
@test lcs_fix_gen_o("AGGTAB", "GXTXAYB") == 4
@test lcs_fix_gen_o("ABC", "CBA") == 1

Test Passed